In [16]:
!pip install -qU gradio python-dotenv langchain-upstage python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import gradio as gr

from langchain_upstage import (
    ChatUpstage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage(streaming=True)

In [217]:
# More general chat
prompt1 = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 발언에서 비속어를 탐지하고, 표준어로 대체합니다. 비속어가 포함된 단어만 수정하고, 다른 단어는 절대로 건드리지 말고 그대로 유지합니다.\
         예시 입력이 주어지면 그 텍스트 뒤에 '에서 비속어를 표준어로 대체해줘'를 붙여서 새로운 텍스트에 대해 동작을 수행해줘."),
        ("human", "꼬우면 니가 차량 크기를 줄이세요. 차선 다 기본적으로 다 먹는 카니발 들고 와서 징징거리면 어쩌란 소리임."),
        ("ai", "마음에 안 들면 니가 차량 크기를 줄이세요. 차선 다 기본적으로 다 먹는 카니발 들고 와서 징징거리면 어떡하란 소리임."),
        ("human", "{message}"),
    ]
)

In [218]:
chain = prompt1 | llm | StrOutputParser()

In [225]:
tmp = chain.invoke("와, 저 잼민이새끼 진짜 인싸 코스프레 오지네")
print(tmp)

와, 저 잼민이새끼 진짜 인싸 코스프레 오지네.


In [ ]:
prompt2 = ChatPromptTemplate.from_messages(
    [
    ("system", "변환하는 문장에 대한 의미는 가능한 비슷하게 유지해줘. 전체적인 문장의 형식은 비슷하게 유지해줘.  내가 주는 문장에 대해서 다른 사람이 불쾌하지 않도록 바꿔줘."),
    ("human", "만나도 짜증 나 헤어져도 아직도 짜증 나 ㅋㅋ 헤어졌다고 해도 안 믿네 어쩌네.. 지금 타이밍에 기자들 엄청 쫓아다닐 텐데 어떻게 만나냐."),
    ("ai", "만나도 짜능 나 헤어져도 아직도 짜증 나 ㅋㅋ 헤어졌다고 해도 안 믿는다고 하시면... 지금 타이밍에 기자들 엄청 취재하고 있을텐데 어떻게 만나겠어요?"),
    ("human", "{message}"),
    ]
) 

In [227]:
chain2 = prompt2 | llm | StrOutputParser()

In [228]:
print(chain2.invoke(tmp))

저 친구는 인싸인 척하는 것 같은데, 좀 과하네.


In [119]:
def chat(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    return chain.invoke({"message": message, "history": history_langchain_format})

In [120]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        title="댓글 순화기",
    )
    chatbot.chatbot.height = 300

In [121]:
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [15]:
import os
from openai import OpenAI
from pprint import pprint

client = OpenAI(
    api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
)
chat_result = client.chat.completions.create(
    model="solar-mini",
    messages=[
        {"role": "system", "content": "너는 텍스트가 객관적인 사실성 텍스트인지 확인해줘."},
        {"role": "user", "content": "뉴진스 노래 잘한다"},
        {
            "role": "assistant",
            "content": "객관적인 사실성 텍스트가 아닙니다. 주관적인 텍스트입니다.",
        },
        {"role": "user", "content": "히틀러는 나쁜 사람이다"},
    ],
)
pprint(chat_result)
print("Message only:")
pprint(chat_result.choices[0].message.content)

ChatCompletion(id='11a452b6-7e42-446c-b3dc-6704a0005781', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='객관적인 사실성 텍스트가 아닙니다. 주관적인 텍스트입니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730517564, model='solar-mini-240612', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=68, total_tokens=82, completion_tokens_details=None, prompt_tokens_details=None))
Message only:
'객관적인 사실성 텍스트가 아닙니다. 주관적인 텍스트입니다.'
